In [12]:
import numpy as np
import pandas as pd
from datetime import datetime as dt

In [21]:
# Remove timezone, city name, lat, lon, visibility, 
data = pd.read_csv("../data/raw/sf_weather_historic.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 409296 entries, 0 to 409295
Data columns (total 28 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   dt                   409296 non-null  int64  
 1   dt_iso               409296 non-null  object 
 2   timezone             409296 non-null  int64  
 3   city_name            409296 non-null  object 
 4   lat                  409296 non-null  float64
 5   lon                  409296 non-null  float64
 6   temp                 409296 non-null  float64
 7   visibility           0 non-null       float64
 8   dew_point            409296 non-null  float64
 9   feels_like           409296 non-null  float64
 10  temp_min             409296 non-null  float64
 11  temp_max             409296 non-null  float64
 12  pressure             409296 non-null  int64  
 13  sea_level            0 non-null       float64
 14  grnd_level           0 non-null       float64
 15  humidity         

In [6]:
new = pd.read_csv("/Users/cyrusnavasca9/sf-aqi-prediction/sf-aqi-prediction/data/openweather_20250909_192855.csv")
new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   weather          1 non-null      object 
 1   base             1 non-null      object 
 2   visibility       1 non-null      int64  
 3   dt               1 non-null      int64  
 4   timezone         1 non-null      int64  
 5   id               1 non-null      int64  
 6   name             1 non-null      object 
 7   cod              1 non-null      int64  
 8   coord.lon        1 non-null      float64
 9   coord.lat        1 non-null      float64
 10  main.temp        1 non-null      float64
 11  main.feels_like  1 non-null      float64
 12  main.temp_min    1 non-null      float64
 13  main.temp_max    1 non-null      float64
 14  main.pressure    1 non-null      int64  
 15  main.humidity    1 non-null      int64  
 16  main.sea_level   1 non-null      int64  
 17  main.grnd_level  1 n

In [22]:
# Only including relevant columns
data = data[["dt", "temp", "feels_like", "temp_min", "temp_max", "pressure",
"humidity", "wind_speed", "wind_deg", "wind_gust", "clouds_all"]]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 409296 entries, 0 to 409295
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   dt          409296 non-null  int64  
 1   temp        409296 non-null  float64
 2   feels_like  409296 non-null  float64
 3   temp_min    409296 non-null  float64
 4   temp_max    409296 non-null  float64
 5   pressure    409296 non-null  int64  
 6   humidity    409296 non-null  int64  
 7   wind_speed  409296 non-null  float64
 8   wind_deg    409296 non-null  int64  
 9   wind_gust   33310 non-null   float64
 10  clouds_all  409296 non-null  int64  
dtypes: float64(6), int64(5)
memory usage: 34.3 MB


In [ ]:
date = dt.strptime("09-13-2023", "%m-%d-%Y").timestamp()
data = data[data["dt"] >= date]
data.sort_values("dt", inplace=True)
data["wind_gust"] = data["wind_gust"].fillna(0)

,dt,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,wind_gust,clouds_all
391831,1694588400,61.92,62.04,59.36,64.17,1013,90,1.01,188,4.00,2
391832,1694592000,61.79,61.84,60.58,63.39,1013,89,4.92,249,NaN,3
391833,1694595600,60.93,60.91,59.76,62.46,1013,89,4.88,246,NaN,8
391834,1694599200,59.41,59.38,58.55,60.51,1013,92,5.50,268,NaN,45
391835,1694602800,58.33,58.19,57.42,59.32,1013,92,4.83,268,NaN,47
...,...,...,...,...,...,...,...,...,...,...,...
409291,1757444400,71.24,71.73,66.61,73.74,1015,78,3.00,254,8.01,45
409292,1757448000,72.05,72.57,68.04,74.08,1014,77,4.00,204,8.99,72
409293,1757451600,71.82,72.41,68.04,74.08,1014,79,3.00,231,8.01,72
409294,1757455200,72.05,72.63,68.04,74.08,1014,78,3.00,224,10.00,72


In [ ]:
# Create mapping from recent data columns to historical data columns
def create_mergeable_dataset(historical_df, recent_df):
    """
    Create a new dataset with only columns that can merge with recent data
    """
    
    # Define column mappings: recent_column -> historical_column
    column_mapping = {
        'dt': 'dt',
        'timezone': 'timezone', 
        #'visibility': 'visibility',
        'main.temp': 'temp',
        'main.feels_like': 'feels_like',
        'main.temp_min': 'temp_min',
        'main.temp_max': 'temp_max',
        'main.pressure': 'pressure',
        'main.humidity': 'humidity',
        'wind.speed': 'wind_speed',
        'wind.deg': 'wind_deg',
        'wind.gust': 'wind_gust',
        'clouds.all': 'clouds_all',
        #'coord.lat': 'lat',
        #'coord.lon': 'lon'
    }
    
    # Get columns that exist in both datasets
    available_mappings = {}
    for recent_col, hist_col in column_mapping.items():
        if recent_col in recent_df.columns and hist_col in historical_df.columns:
            available_mappings[recent_col] = hist_col
    
    print("=== AVAILABLE COLUMNS FOR MERGING ===")
    for recent_col, hist_col in available_mappings.items():
        print(f"{recent_col} -> {hist_col}")
    
    # Create new dataset with only mergeable columns
    mergeable_columns = list(available_mappings.values())
    data_mergeable = historical_df[mergeable_columns].copy()
    
    print(f"\nCreated dataset with {len(data_mergeable)} rows and {len(mergeable_columns)} columns")
    print(f"Columns: {mergeable_columns}")
    
    return data_mergeable, available_mappings

# Create the mergeable dataset
data_mergeable, mappings = create_mergeable_dataset(data, new)


In [4]:
data.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,283996800,1979-01-01 00:00:00 +0000 UTC,-28800,San Francisco,37.774929,-122.419415,50.32,NaN,33.39,47.50,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01d
1,284000400,1979-01-01 01:00:00 +0000 UTC,-28800,San Francisco,37.774929,-122.419415,51.80,NaN,34.75,49.12,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01d
2,284004000,1979-01-01 02:00:00 +0000 UTC,-28800,San Francisco,37.774929,-122.419415,50.74,NaN,34.72,48.06,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n
3,284007600,1979-01-01 03:00:00 +0000 UTC,-28800,San Francisco,37.774929,-122.419415,42.53,NaN,32.74,39.63,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n
4,284011200,1979-01-01 04:00:00 +0000 UTC,-28800,San Francisco,37.774929,-122.419415,39.52,NaN,31.66,37.29,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n


In [7]:
new.head()

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 803, 'main': 'Clouds', 'description': ...",stations,10000,1757446135,-25200,5330810,Brisbane,200,-122.3988,37.715,...,1009,8.99,254,13,75,2,2079712,US,1757425611,1757471226
